# 智能体经典范式构建

## 环境准备与基础工具定义

### 1.1 安装依赖库

In [1]:
# 检查安装的库
#!pip show openai python-dotenv
# 安装依赖库
#!pip install openai python-dotenv


### 1.2 配置 API 密钥

将模型服务的相关信息（模型ID、API密钥、服务地址）统一配置在环境变量中。

- 在项目根目录下，创建一个名为 .env 的文件。
- 在该文件中，添加以下内容。根据自己的需要，将其指向 OpenAI 官方服务，或任何兼容 OpenAI 接口的本地/第三方服务。
```
# .env file
LLM_API_KEY="YOUR-API-KEY"
LLM_MODEL_ID="YOUR-MODEL"
LLM_BASE_URL="YOUR-URL"
```

### 1.3 封装基础 LLM 调用函数


In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Optional

# 加载 .env 文件 (确保能读取到根目录的 .env)
# 假设当前运行目录在项目根目录，或者显式指定 .env 路径
load_dotenv() 

class HelloAgentsLLM:
    """
    适配 Datawhale Hello Agents 教程的 LLM 客户端。
    支持自动加载 .env 中的通用配置，也支持传入特定参数。
    """
    def __init__(self, model: str = None, apiKey: str = None, baseUrl: str = None, timeout: int = None):
        # 1. 尝试使用传入参数
        # 2. 尝试读取教程标准的通用环境变量 (LLM_*)
        # 3. 兜底：尝试读取项目中已有的特定厂商环境变量 (如 QWEN_*) 以方便直接使用
        
        self.model = model or os.getenv("LLM_MODEL_ID") or "qwen-turbo" # 默认示例
        
        # 自动适配 Key 和 BaseURL
        self.api_key = apiKey or os.getenv("LLM_API_KEY")
        self.base_url = baseUrl or os.getenv("LLM_BASE_URL")
        self.timeout = timeout or int(os.getenv("LLM_TIMEOUT", 60))
        
        # 如果没有通用的 LLM_API_KEY，尝试自动通过模型名匹配已有的 Key (可选优化)
        if not self.api_key:
            if "qwen" in self.model.lower():
                self.api_key = os.getenv("QWEN_API_KEY")
                self.base_url = os.getenv("QWEN_BASE_URL")
            elif "deepseek" in self.model.lower():
                self.api_key = os.getenv("DEEPSEEK_API_KEY")
                self.base_url = os.getenv("DEEPSEEK_BASE_URL")
            # ... 其他模型适配
            
        if not self.api_key:
            raise ValueError("未找到 API Key。请在 .env 中配置 LLM_API_KEY 或特定的模型 Key。")

        # 初始化 OpenAI 客户端
        self.client = OpenAI(
            api_key=self.api_key,
            base_url=self.base_url,
            timeout=self.timeout
        )

    def think(self, messages: List[Dict[str, str]], temperature: float = 0.7, stream: bool = False) -> str:
        """
        核心方法：发送消息历史并获取回复
        :param stream: 是否开启流式输出 (打印到控制台)
        """
        print(f"🧠 正在调用 {self.model} 模型...")
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=stream 
            )
            
            if not stream:
                # 非流式：直接返回
                print("✅ 大语言模型响应成功:", flush=True)
                return response.choices[0].message.content
            else:
                # 流式处理逻辑
                collected_content = []
                print("✅ 大语言模型响应成功:", flush=True)
                
                for chunk in response:
                    content = chunk.choices[0].delta.content or ""                   
                    print(content, end="", flush=True)
                    collected_content.append(content)
                print()  # 在流式输出结束后换行
                return "".join(collected_content)

        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return f"Error calling LLM: {str(e)}"

In [6]:
# 1. 实例化 
llm = HelloAgentsLLM(model="mimo-v2-flash") 

# 或者指定其他模型
# llm = HelloAgentsLLM(model="deepseek-chat")

# 2. 构造消息
messages = [
    {"role": "user", "content": "你好，华为最新的手机是哪一款？它的主要卖点是什么？"}
]

# 3. 调用 (非流式，直接拿到结果)
response = llm.think(messages, stream=False)
if response:
            print(response) 
            print("\n\n--- 完整模型响应 ---")
            

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
你好！

截至到目前（2024年5月），华为最新、最受关注的旗舰手机系列是 **HUAWEI Pura 70 系列**（包括 Pura 70、Pura 70 Pro、Pura 70 Ultra 等型号）。

此外，华为在 2023 年底发布的 **Mate 60 系列** 依然保持着极高的热度，也是目前主力销售的旗舰机型。

以下是这两款系列手机的主要卖点分析：

### 1. HUAWEI Pura 70 系列 (最新影像旗舰)
这是华为将 P 系列升级为 Pura 系列后的首款产品，主打**影像**和**设计**。

*   **极致的影像能力（Ultra版）：**
    *   **伸缩镜头技术：** Pura 70 Ultra 首次采用了可伸缩的相机模组，通过旋转导向结构，实现了手机摄影的物理突破。
    *   **1英寸大底传感器：** 主摄采用了1英寸超大底传感器（行业首发），配合 F1.6 物理光圈，进光量极大，夜景和抓拍能力极强。
    *   **超高速风驰闪拍：** 升级了XD Motion运动引擎，可以轻松抓拍极高速运动的物体（如行驶的汽车、奔跑的动物）。
*   **独特的美学设计：**
    *   **三角形相机岛：** 极具辨识度的三角形后摄模组设计。
    *   **配色与材质：** 引入了“星芒纹”设计以及高定的刺绣压纹工艺，配色非常时尚年轻。
*   **灵犀通信与北斗卫星消息：**
    *   全系支持双向北斗卫星消息（Ultra版支持北斗卫星图片消息），在无信号区域也能保持联系。
*   **性能与系统：**
    *   搭载 HarmonyOS 4.2，系统流畅度极高，AI功能（如AI消除、AI云增强）非常实用。

### 2. HUAWEI Mate 60 系列 (均衡商务旗舰)
虽然发布稍早，但它代表了华为的重大回归，是目前依然极具竞争力的“当家花旦”。

*   **麒麟芯片回归：**
    *   搭载自研的麒麟 9000S 芯片，支持 5G 网络（实际显示为4G信号，但网速达到5G标准），标志着华为解决了核心芯片供应问题，性能强劲且优化极佳。
*   **卫星通话技术：**
    *   **Mate 60 P

In [58]:
# 1. 实例化
llm = HelloAgentsLLM(model="mimo-v2-flash") 

# 或者指定其他模型
# llm = HelloAgentsLLM(model="deepseek-chat")

# 2. 构造消息
messages = [
    {"role": "user", "content": "你好，请介绍一下你自己"}
]

# 3. 调用 (流式)
print("--- 调用LLM ---")
response = llm.think(messages, stream=True)
if response:
            print("\n\n--- 完整模型响应 ---")
            print(response)

--- 调用LLM ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
你好！我是 MiMo，由小米大模型 Core 团队开发的人工智能助手。我的使命是为用户提供有帮助、安全、友好的回答。

我可以帮你做很多事情，比如：
-   **回答问题**：无论是科学知识、历史典故，还是生活常识，我都会尽力为你解答。
-   **获取资讯**：我可以帮你了解最新的科技动态、热门事件等。
-   **创作与写作**：需要写一篇文案、构思一个故事，或者润色一段文字，我都可以协助你。
-   **生活助手**：旅行攻略、美食推荐、学习计划制定，我也是你的得力助手。
-   **编程与技术**：如果你在学习编程或遇到技术难题，我可以提供代码示例和思路。

总之，无论你有什么想了解的或者需要帮助的，随时都可以告诉我。很高兴认识你，有什么可以帮你的吗？❌ 调用LLM API时发生错误: list index out of range


--- 完整模型响应 ---
Error calling LLM: list index out of range


## ReAct

### 2.1 ReAct 的工作流程

### 2.2 工具的定义与实现

In [14]:
# 检查是否已安装 google-search-results
# !pip show google-search-results
# 安装依赖
#!pip install google-search-results


In [7]:
from serpapi import SerpApiClient

def search(query: str) -> str:
    """
    一个基于SerpApi的实战网页搜索引擎工具。
    它会智能地解析搜索结果，优先返回直接答案或知识图谱信息。
    """
    print(f"🔍 正在执行 [SerpApi] 网页搜索: {query}")
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "错误:SERPAPI_API_KEY 未在 .env 文件中配置。"

        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "gl": "cn",  # 国家代码
            "hl": "zh-cn", # 语言代码
        }
        
        client = SerpApiClient(params)
        results = client.get_dict()
        
        # 智能解析:优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"

In [8]:
from typing import Dict, Any

class ToolExecutor:
    """
    一个工具执行器，负责管理和执行工具。
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name: str, description: str, func: callable):
        """
        向工具箱中注册一个新工具。
        """
        if name in self.tools:
            print(f"警告:工具 '{name}' 已存在，将被覆盖。")
        self.tools[name] = {"description": description, "func": func}
        print(f"工具 '{name}' 已注册。")

    def getTool(self, name: str) -> callable:
        """
        根据名称获取一个工具的执行函数。
        """
        return self.tools.get(name, {}).get("func")

    def getAvailableTools(self) -> str:
        """
        获取所有可用工具的格式化描述字符串。
        """
        return "\n".join([
            f"- {name}: {info['description']}" 
            for name, info in self.tools.items()
        ])


In [9]:
# --- 工具初始化与使用示例 ---
if __name__ == '__main__':
    # 1. 初始化工具执行器
    toolExecutor = ToolExecutor()

    # 2. 注册我们的实战搜索工具
    search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
    toolExecutor.registerTool("Search", search_description, search)
    
    # 3. 打印可用的工具
    print("\n--- 可用的工具 ---")
    print(toolExecutor.getAvailableTools())

    # 4. 智能体的Action调用，这次我们问一个实时性的问题
    print("\n--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---")
    tool_name = "Search"
    tool_input = "英伟达最新的GPU型号是什么"

    tool_function = toolExecutor.getTool(tool_name)
    if tool_function:
        observation = tool_function(tool_input)
        print("--- 观察 (Observation) ---")
        print(observation)
    else:
        print(f"错误:未找到名为 '{tool_name}' 的工具。")
        

工具 'Search' 已注册。

--- 可用的工具 ---
- Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---
🔍 正在执行 [SerpApi] 网页搜索: 英伟达最新的GPU型号是什么
--- 观察 (Observation) ---
[1] 比较GeForce 系列最新一代显卡和前代显卡
比较最新一代RTX 30 系列显卡和前代的RTX 20 系列、GTX 10 和900 系列显卡。查看规格、功能、技术支持等内容。

[2] GeForce 显卡| NVIDIA
探索NVIDIA GeForce 显卡。RTX 50 系列和RTX 40 系列。

[3] 一文彻底读懂：英伟达GPU分类、架构演进和参数解析
Quadro系列是英伟达专业级GPU产品线，针对商业和专业应用领域进行了优化。常见的产品型号如NVIDIA RTX A6000、A5000等。 Quadro GPU具备强大的计算能力、大 ...


### 2.3 ReAct 智能体的编码实现

### 2.4 运行实例与分析

In [59]:
from ReAct import ReActAgent

llm = HelloAgentsLLM()
tool_executor = ToolExecutor()
search_desc = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.registerTool("Search", search_desc, search)
agent = ReActAgent(llm_client=llm, tool_executor=tool_executor)
question = "华为最新的手机是哪一款？它的主要卖点是什么？"
agent.run(question)

工具 'Search' 已注册。

--- 第 1 步 ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
🤔 思考: 用户询问华为最新的手机型号及其主要卖点，这是一个关于科技产品最新动态的事实性问题。由于我的知识库可能不是最新的，我需要使用搜索工具来获取华为最新的手机信息。
🎬 行动: Search[华为最新的手机型号及其主要卖点]
🔍 正在执行 [SerpApi] 网页搜索: 华为最新的手机型号及其主要卖点
👀 观察: [1] 华为手机- 华为官网
探索并选购华为最新手机，了解Mate 系列、Pura 系列、Pocket 系列、nova 系列、畅享系列及相关配件，体验鸿蒙AI 、影像、通信等功能。

[2] 华为nova 15系列突然官宣：三款机型齐发且卖点清晰
智能手机 ... 其区别于同价位机型的关键卖点。 最后则是新机预装全新的鸿蒙OS 6.0系统，带来更流畅的动画、更低的跨设备协同延迟，以及对Wi-Fi 7等新标准的 ...

[3] 2025年华为手机哪一款性价比高？华为手机推荐与市场分析
现在华为手机最大的卖点只剩下鸿蒙HarmonyOS系统，以及饱受争议的品牌信仰。 这里推荐目前值得入手的几款华为系列手机，根据不同预算自行选择：. 华为目前最受欢迎，也是搭载 ...

--- 第 2 步 ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
🤔 思考: 用户询问华为最新的手机是哪一款以及它的主要卖点。从搜索结果来看，华为nova 15系列刚刚官宣，这很可能是最新的手机型号。我需要进一步了解这款手机的具体信息，包括发布日期和主要卖点。同时，我也需要确认是否还有其他更新的型号。让我先查看华为官网和nova 15系列的详细信息。
🎬 行动: Search[华为nova 15系列详细规格和卖点]
🔍 正在执行 [SerpApi] 网页搜索: 华为nova 15系列详细规格和卖点
👀 观察: [1] 华为nova 15/Pro/Ultra官网上架：全系外观、储存方案公布
nova 15 Pro和nova 15 Ultra则是升级了全新外观，系列首次采用横向大矩阵模组，预计都会搭载红枫原色镜头。 华为nova 15/Pro/Ultra官网上架：全系外观、储存 ...

[2] 华为nova 

'华为最新的手机是华为nova 15系列（包括nova 15、nova 15 Pro和nova 15 Ultra）。主要卖点包括：1. 预装全新的鸿蒙OS 6.0系统，带来更流畅的动画、更低的跨设备协同延迟，并支持Wi-Fi 7等新标准；2. 性能方面，nova 15搭载麒麟8系处理器，Pro和Ultra版本则升级为麒麟9系芯片，提供更强的性能；3. 影像系统采用横向大矩阵模组设计，并首次搭载红枫原色镜头，提升拍照效果；4. 外观设计全面焕新，采用横向立体堆叠的双圆环设计、挺阔直屏与轻薄机身，具有高辨识度。'

## Plan and solve

### 3.1 Plan-and-Solve 的工作原理

### 3.2 规划阶段

In [3]:
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划,```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""

In [4]:
# 假定 llm_client.py 中的 HelloAgentsLLM 类已经定义好
# from llm_client import HelloAgentsLLM

class Planner:
    def __init__(self, llm_client):
        self.llm_client = llm_client

    def plan(self, question: str) -> list[str]:
        """
        根据用户问题生成一个行动计划。
        """
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        
        # 为了生成计划，我们构建一个简单的消息列表
        messages = [{"role": "user", "content": prompt}]
        
        print("--- 正在生成计划 ---")
        # 使用流式输出来获取完整的计划
        response_text = self.llm_client.think(messages=messages) or ""
        
        print(f"✅ 计划已生成:\n{response_text}")
        
        # 解析LLM输出的列表字符串
        try:
            # 找到```python和```之间的内容
            plan_str = response_text.split("```python")[1].split("```")[0].strip()
            # 使用ast.literal_eval来安全地执行字符串，将其转换为Python列表
            plan = ast.literal_eval(plan_str)
            return plan if isinstance(plan, list) else []
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"❌ 解析计划时出错: {e}")
            print(f"原始响应: {response_text}")
            return []
        except Exception as e:
            print(f"❌ 解析计划时发生未知错误: {e}")
            return []

### 3.3 执行器与状态管理

In [5]:
EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请仅输出针对“当前步骤”的回答:
"""

In [6]:
class Executor:
    def __init__(self, llm_client):
        self.llm_client = llm_client

    def execute(self, question: str, plan: list[str]) -> str:
        """
        根据计划，逐步执行并解决问题。
        """
        history = "" # 用于存储历史步骤和结果的字符串
        
        print("\n--- 正在执行计划 ---")
        
        for i, step in enumerate(plan):
            print(f"\n-> 正在执行步骤 {i+1}/{len(plan)}: {step}")
            
            prompt = EXECUTOR_PROMPT_TEMPLATE.format(
                question=question,
                plan=plan,
                history=history if history else "无", # 如果是第一步，则历史为空
                current_step=step
            )
            
            messages = [{"role": "user", "content": prompt}]
            
            response_text = self.llm_client.think(messages=messages) or ""
            
            # 更新历史记录，为下一步做准备
            history += f"步骤 {i+1}: {step}\n结果: {response_text}\n\n"
            
            print(f"✅ 步骤 {i+1} 已完成，结果: {response_text}")

        # 循环结束后，最后一步的响应就是最终答案
        final_answer = response_text
        return final_answer

In [7]:
class PlanAndSolveAgent:
    def __init__(self, llm_client):
        """
        初始化智能体，同时创建规划器和执行器实例。
        """
        self.llm_client = llm_client
        self.planner = Planner(self.llm_client)
        self.executor = Executor(self.llm_client)

    def run(self, question: str):
        """
        运行智能体的完整流程:先规划，后执行。
        """
        print(f"\n--- 开始处理问题 ---\n问题: {question}")
        
        # 1. 调用规划器生成计划
        plan = self.planner.plan(question)
        
        # 检查计划是否成功生成
        if not plan:
            print("\n--- 任务终止 --- \n无法生成有效的行动计划。")
            return

        # 2. 调用执行器执行计划
        final_answer = self.executor.execute(question, plan)
        
        print(f"\n--- 任务完成 ---\n最终答案: {final_answer}")

### 3.4 运行实例与分析

In [10]:
llm_client = HelloAgentsLLM()
agent = PlanAndSolveAgent(llm_client)
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
agent.run(question)    


--- 开始处理问题 ---
问题: 一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？
--- 正在生成计划 ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 计划已生成:
```python
["计算周一卖出的苹果数量", "计算周二卖出的苹果数量（周一的两倍）", "计算周三卖出的苹果数量（比周二少5个）", "将这三天的苹果数量相加得到总数"]
```

--- 正在执行计划 ---

-> 正在执行步骤 1/4: 计算周一卖出的苹果数量
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 1 已完成，结果: 15

-> 正在执行步骤 2/4: 计算周二卖出的苹果数量（周一的两倍）
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 2 已完成，结果: 15 * 2 = 30

-> 正在执行步骤 3/4: 计算周三卖出的苹果数量（比周二少5个）
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 3 已完成，结果: 30 - 5 = 25

-> 正在执行步骤 4/4: 将这三天的苹果数量相加得到总数
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 4 已完成，结果: 15 + 30 + 25 = 70

--- 任务完成 ---
最终答案: 15 + 30 + 25 = 70


In [9]:
from Plan_and_solve import PlanAndSolveAgent

llm_client = HelloAgentsLLM()
agent = PlanAndSolveAgent(llm_client)
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
agent.run(question)       


--- 开始处理问题 ---
问题: 一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？
--- 正在生成计划 ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 计划已生成:
```python
["计算周二卖出的苹果数量，即周一的两倍：15 * 2", "计算周三卖出的苹果数量，即周二的数量减去5", "计算三天总共卖出的苹果数量，将周一、周二和周三的数量相加"]
```

--- 正在执行计划 ---

-> 正在执行步骤 1/3: 计算周二卖出的苹果数量，即周一的两倍：15 * 2
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 1 已完成，结果: 30

-> 正在执行步骤 2/3: 计算周三卖出的苹果数量，即周二的数量减去5
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 2 已完成，结果: 30 - 5 = 25

-> 正在执行步骤 3/3: 计算三天总共卖出的苹果数量，将周一、周二和周三的数量相加
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
✅ 步骤 3 已完成，结果: 15 + 30 + 25 = 70

--- 任务完成 ---
最终答案: 15 + 30 + 25 = 70


## Reflection

### 4.1 Reflection 机制的核心思想

Reflection 机制的灵感来源于人类的学习过程：我们完成初稿后会进行校对，解出数学题后会进行验算。这一思想在多个研究中得到了体现，例如 Shinn, Noah 在2023年提出的 Reflexion 框架<sup>[3]</sup>。其核心工作流程可以概括为一个简洁的三步循环：<strong>执行 -> 反思 -> 优化</strong>。

1. <strong>执行 (Execution)</strong>：首先，智能体使用我们熟悉的方法（如 ReAct 或 Plan-and-Solve）尝试完成任务，生成一个初步的解决方案或行动轨迹。这可以看作是“初稿”。
2. <strong>反思 (Reflection)</strong>：接着，智能体进入反思阶段。它会调用一个独立的、或者带有特殊提示词的大语言模型实例，来扮演一个“评审员”的角色。这个“评审员”会审视第一步生成的“初稿”，并从多个维度进行评估，例如：
   - <strong>事实性错误</strong>：是否存在与常识或已知事实相悖的内容？
   - <strong>逻辑漏洞</strong>：推理过程是否存在不连贯或矛盾之处？
   - <strong>效率问题</strong>：是否有更直接、更简洁的路径来完成任务？
   - <strong>遗漏信息</strong>：是否忽略了问题的某些关键约束或方面？ 根据评估，它会生成一段结构化的<strong>反馈 (Feedback)</strong>，指出具体的问题所在和改进建议。
3. <strong>优化 (Refinement)</strong>：最后，智能体将“初稿”和“反馈”作为新的上下文，再次调用大语言模型，要求它根据反馈内容对初稿进行修正，生成一个更完善的“修订稿”。

与前两种范式相比，Reflection 的价值在于：

- 它为智能体提供了一个内部纠错回路，使其不再完全依赖于外部工具的反馈（ReAct 的 Observation），从而能够修正更高层次的逻辑和策略错误。
- 它将一次性的任务执行，转变为一个持续优化的过程，显著提升了复杂任务的最终成功率和答案质量。
- 它为智能体构建了一个临时的<strong>“短期记忆”</strong>。整个“执行-反思-优化”的轨迹形成了一个宝贵的经验记录，智能体不仅知道最终答案，还记得自己是如何从有缺陷的初稿迭代到最终版本的。更进一步，这个记忆系统还可以是<strong>多模态的</strong>，允许智能体反思和修正文本以外的输出（如代码、图像等），为构建更强大的多模态智能体奠定了基础。

### 4.2 案例设定与记忆模块设计

- <strong>代码生成与迭代优化</strong>

这一步的目标任务是：“编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。”

这个任务是检验 Reflection 机制的绝佳场景：

1. <strong>存在明确的优化路径</strong>：大语言模型初次生成的代码很可能是一个简单但效率低下的递归实现。
2. <strong>反思点清晰</strong>：可以通过反思发现其“时间复杂度过高”或“存在重复计算”的问题。
3. <strong>优化方向明确</strong>：可以根据反馈，将其优化为更高效的迭代版本或使用备忘录模式的版本。

Reflection 的核心在于迭代，而迭代的前提是能够记住之前的尝试和获得的反馈。因此，一个“短期记忆”模块是实现该范式的必需品。这个记忆模块将负责存储每一次“执行-反思”循环的完整轨迹。

In [2]:
from typing import List, Dict, Any, Optional

class Memory:
    """
    一个简单的短期记忆模块，用于存储智能体的行动与反思轨迹。
    """

    def __init__(self):
        """
        初始化一个空列表来存储所有记录。
        """
        self.records: List[Dict[str, Any]] = []

    def add_record(self, record_type: str, content: str):
        """
        向记忆中添加一条新记录。

        参数:
        - record_type (str): 记录的类型 ('execution' 或 'reflection')。
        - content (str): 记录的具体内容 (例如，生成的代码或反思的反馈)。
        """
        record = {"type": record_type, "content": content}
        self.records.append(record)
        print(f"📝 记忆已更新，新增一条 '{record_type}' 记录。")

    def get_trajectory(self) -> str:
        """
        将所有记忆记录格式化为一个连贯的字符串文本，用于构建提示词。
        """
        trajectory_parts = []
        for record in self.records:
            if record['type'] == 'execution':
                trajectory_parts.append(f"--- 上一轮尝试 (代码) ---\n{record['content']}")
            elif record['type'] == 'reflection':
                trajectory_parts.append(f"--- 评审员反馈 ---\n{record['content']}")
        
        return "\n\n".join(trajectory_parts)

    def get_last_execution(self) -> Optional[str]:
        """
        获取最近一次的执行结果 (例如，最新生成的代码)。
        如果不存在，则返回 None。
        """
        for record in reversed(self.records):
            if record['type'] == 'execution':
                return record['content']
        return None

这个 `Memory` 类的设计比较简洁，主体是这样的：

- 使用一个列表 `records` 来按顺序存储每一次的行动和反思。
- `add_record` 方法负责向记忆中添加新的条目。
- `get_trajectory` 方法是核心，它将记忆轨迹“序列化”成一段文本，可以直接插入到后续的提示词中，为模型的反思和优化提供完整的上下文。
- `get_last_execution` 方便我们获取最新的“初稿”以供反思。

### 4.3 Reflection 智能体的编码实现

#### （1）提示词设计

与之前的范式不同，Reflection 机制需要多个不同角色的提示词来协同工作。

1. <strong>初始执行提示词 (Execution Prompt)</strong> ：这是智能体首次尝试解决问题的提示词，内容相对直接，只要求模型完成指定任务。

In [3]:
INITIAL_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。请根据以下要求，编写一个Python函数。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。

要求: {task}

请直接输出代码，不要包含任何额外的解释。
"""

2. <strong>反思提示词 (Reflection Prompt)</strong> ：这个提示词是 Reflection 机制的灵魂。它指示模型扮演“代码评审员”的角色，对上一轮生成的代码进行批判性分析，并提供具体的、可操作的反馈。

In [4]:
REFLECT_PROMPT_TEMPLATE = """
你是一位极其严格的代码评审专家和资深算法工程师，对代码的性能有极致的要求。
你的任务是审查以下Python代码，并专注于找出其在<strong>算法效率</strong>上的主要瓶颈。

# 原始任务:
{task}

# 待审查的代码:
```python
{code}
```

请分析该代码的时间复杂度，并思考是否存在一种<strong>算法上更优</strong>的解决方案来显著提升性能。
如果存在，请清晰地指出当前算法的不足，并提出具体的、可行的改进算法建议（例如，使用筛法替代试除法）。
如果代码在算法层面已经达到最优，才能回答“无需改进”。

请直接输出你的反馈，不要包含任何额外的解释。
"""

3. <strong>优化提示词 (Refinement Prompt)</strong> ：当收到反馈后，这个提示词将引导模型根据反馈内容，对原有代码进行修正和优化。

In [5]:
REFINE_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。你正在根据一位代码评审专家的反馈来优化你的代码。

# 原始任务:
{task}

# 你上一轮尝试的代码:
{last_code_attempt}
评审员的反馈：
{feedback}

请根据评审员的反馈，生成一个优化后的新版本代码。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。
请直接输出优化后的代码，不要包含任何额外的解释。
"""

#### （2）智能体封装与实现

现在，我们将这套提示词逻辑和 `Memory` 模块整合到 `ReflectionAgent` 类中。

In [6]:
# 假设 llm_client.py 和 memory.py 已定义
# from llm_client import HelloAgentsLLM
# from memory import Memory

class ReflectionAgent:
    def __init__(self, llm_client, max_iterations=3):
        self.llm_client = llm_client
        self.memory = Memory()
        self.max_iterations = max_iterations

    def run(self, task: str):
        print(f"\n--- 开始处理任务 ---\n任务: {task}")

        # --- 1. 初始执行 ---
        print("\n--- 正在进行初始尝试 ---")
        initial_prompt = INITIAL_PROMPT_TEMPLATE.format(task=task)
        initial_code = self._get_llm_response(initial_prompt)
        self.memory.add_record("execution", initial_code)

        # --- 2. 迭代循环:反思与优化 ---
        for i in range(self.max_iterations):
            print(f"\n--- 第 {i+1}/{self.max_iterations} 轮迭代 ---")

            # a. 反思
            print("\n-> 正在进行反思...")
            last_code = self.memory.get_last_execution()
            reflect_prompt = REFLECT_PROMPT_TEMPLATE.format(task=task, code=last_code)
            feedback = self._get_llm_response(reflect_prompt)
            self.memory.add_record("reflection", feedback)

            # b. 检查是否需要停止
            if "无需改进" in feedback:
                print("\n✅ 反思认为代码已无需改进，任务完成。")
                break

            # c. 优化
            print("\n-> 正在进行优化...")
            refine_prompt = REFINE_PROMPT_TEMPLATE.format(
                task=task,
                last_code_attempt=last_code,
                feedback=feedback
            )
            refined_code = self._get_llm_response(refine_prompt)
            self.memory.add_record("execution", refined_code)
        
        final_code = self.memory.get_last_execution()
        print(f"\n--- 任务完成 ---\n最终生成的代码:\n```python\n{final_code}\n```")
        return final_code

    def _get_llm_response(self, prompt: str) -> str:
        """一个辅助方法，用于调用LLM并获取完整的流式响应。"""
        messages = [{"role": "user", "content": prompt}]
        response_text = self.llm_client.think(messages=messages) or ""
        return response_text

### 4.4 运行实例与分析

#### （1）demo 演示

In [ ]:
# 1. 初始化LLM客户端 (请确保你的 .env 和 llm_client.py 文件配置正确)
try:
    llm_client = HelloAgentsLLM()
except Exception as e:
    print(f"初始化LLM客户端时出错: {e}")
    exit()

In [8]:
# 2. 初始化 Reflection 智能体，设置最多迭代2轮
agent = ReflectionAgent(llm_client, max_iterations=2)

In [9]:
# 3. 定义任务并运行智能体
task = "编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。"
agent.run(task)


--- 开始处理任务 ---
任务: 编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。

--- 正在进行初始尝试 ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
📝 记忆已更新，新增一条 'execution' 记录。

--- 第 1/2 轮迭代 ---

-> 正在进行反思...
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
📝 记忆已更新，新增一条 'reflection' 记录。

✅ 反思认为代码已无需改进，任务完成。

--- 任务完成 ---
最终生成的代码:
```python
```python
from typing import List


def find_primes(n: int) -> List[int]:
    """
    找出1到n之间所有的素数 (prime numbers)。

    参数:
        n (int): 一个正整数，表示查找范围的上限。

    返回:
        List[int]: 一个包含1到n之间所有素数的列表。
                   如果n小于2，则返回空列表。
    """
    if n < 2:
        return []

    # 初始化标记数组，默认所有数都是素数
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False  # 0和1不是素数

    # 使用埃拉托斯特尼筛法 (Sieve of Eratosthenes)
    # 只需要遍历到 sqrt(n)
    for i in range(2, int(n**0.5) + 1):
        if sieve[i]:
            # 标记 i 的倍数为非素数
            # 从 i*i 开始标记，因为小于 i*i 的倍数已被更小的素数标记过了
            for j in range(i * i, n + 1, i):
                sieve[j] = False

    # 收集所有标记为 True 的索引
    primes = [n

'```python\nfrom typing import List\n\n\ndef find_primes(n: int) -> List[int]:\n    """\n    找出1到n之间所有的素数 (prime numbers)。\n\n    参数:\n        n (int): 一个正整数，表示查找范围的上限。\n\n    返回:\n        List[int]: 一个包含1到n之间所有素数的列表。\n                   如果n小于2，则返回空列表。\n    """\n    if n < 2:\n        return []\n\n    # 初始化标记数组，默认所有数都是素数\n    sieve = [True] * (n + 1)\n    sieve[0] = sieve[1] = False  # 0和1不是素数\n\n    # 使用埃拉托斯特尼筛法 (Sieve of Eratosthenes)\n    # 只需要遍历到 sqrt(n)\n    for i in range(2, int(n**0.5) + 1):\n        if sieve[i]:\n            # 标记 i 的倍数为非素数\n            # 从 i*i 开始标记，因为小于 i*i 的倍数已被更小的素数标记过了\n            for j in range(i * i, n + 1, i):\n                sieve[j] = False\n\n    # 收集所有标记为 True 的索引\n    primes = [num for num, is_prime in enumerate(sieve) if is_prime]\n    \n    return primes\n```'

In [10]:
from typing import List


def find_primes(n: int) -> List[int]:
    """
    找出1到n之间所有的素数 (prime numbers)。

    参数:
        n (int): 一个正整数，表示查找范围的上限。

    返回:
        List[int]: 一个包含1到n之间所有素数的列表。
                   如果n小于2，则返回空列表。
    """
    if n < 2:
        return []

    # 初始化标记数组，默认所有数都是素数
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False  # 0和1不是素数

    # 使用埃拉托斯特尼筛法 (Sieve of Eratosthenes)
    # 只需要遍历到 sqrt(n)
    for i in range(2, int(n**0.5) + 1):
        if sieve[i]:
            # 标记 i 的倍数为非素数
            # 从 i*i 开始标记，因为小于 i*i 的倍数已被更小的素数标记过了
            for j in range(i * i, n + 1, i):
                sieve[j] = False

    # 收集所有标记为 True 的索引
    primes = [num for num, is_prime in enumerate(sieve) if is_prime]
    
    return primes

In [11]:
print(find_primes(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


#### （2）import 演示

In [15]:
from Reflection import ReflectionAgent

llm_client = HelloAgentsLLM()
agent = ReflectionAgent(llm_client, max_iterations=2)
task = "编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。"
agent.run(task)


--- 开始处理任务 ---
任务: 编写一个Python函数，找出1到n之间所有的素数 (prime numbers)。

--- 正在进行初始尝试 ---
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
📝 记忆已更新，新增一条 'execution' 记录。

--- 第 1/2 轮迭代 ---

-> 正在进行反思...
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
📝 记忆已更新，新增一条 'reflection' 记录。

-> 正在进行优化...
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
📝 记忆已更新，新增一条 'execution' 记录。

--- 第 2/2 轮迭代 ---

-> 正在进行反思...
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功:
📝 记忆已更新，新增一条 'reflection' 记录。

✅ 反思认为代码已无需改进，任务完成。

--- 任务完成 ---
最终生成的代码:
```python
def find_primes(n: int) -> list[int]:
    """
    找出1到n之间所有的素数 (prime numbers)。

    使用优化的埃拉托斯特尼筛法 (Optimized Sieve of Eratosthenes)。
    该实现仅处理奇数，将内存占用减少约50%，并消除偶数检查的开销。

    参数:
        n (int): 一个正整数，表示查找素数的上限（包含n）。

    返回:
        list[int]: 一个包含所有小于等于n的素数的列表。
                   如果n小于2，则返回空列表。
    """
    if n < 2:
        return []

    # 特殊情况：如果上限小于3，直接返回 [2]
    if n == 2:
        return [2]

    # 仅处理奇数：
    # 索引 i 对应数值 2*i + 1
    # 数组大小：(n - 1) // 2，对应从 3 到 n 的奇数
    size = (n - 1) // 2
    

'```python\ndef find_primes(n: int) -> list[int]:\n    """\n    找出1到n之间所有的素数 (prime numbers)。\n\n    使用优化的埃拉托斯特尼筛法 (Optimized Sieve of Eratosthenes)。\n    该实现仅处理奇数，将内存占用减少约50%，并消除偶数检查的开销。\n\n    参数:\n        n (int): 一个正整数，表示查找素数的上限（包含n）。\n\n    返回:\n        list[int]: 一个包含所有小于等于n的素数的列表。\n                   如果n小于2，则返回空列表。\n    """\n    if n < 2:\n        return []\n\n    # 特殊情况：如果上限小于3，直接返回 [2]\n    if n == 2:\n        return [2]\n\n    # 仅处理奇数：\n    # 索引 i 对应数值 2*i + 1\n    # 数组大小：(n - 1) // 2，对应从 3 到 n 的奇数\n    size = (n - 1) // 2\n    is_prime = [True] * size\n\n    # 只需筛选到 sqrt(n)\n    limit = int(n**0.5)\n\n    # 遍历奇数对应的索引\n    # 对应的数值为 3, 5, 7...\n    # 索引范围：从 1 (对应3) 到 (limit - 1) // 2\n    for i in range(1, (limit - 1) // 2 + 1):\n        if is_prime[i]:\n            # 当前素数 p = 2*i + 1\n            p = 2 * i + 1\n            \n            # 标记 p 的倍数（均为奇数）：p*p, p*(p+2)...\n            # p*p 对应的索引：(p*p - 1) // 2\n            # 步长：2*p (因为只标记奇数倍，偶数倍已隐式跳过)\n            start_index =

In [16]:
def find_primes(n: int) -> list[int]:
    """
    找出1到n之间所有的素数 (prime numbers)。

    使用优化的埃拉托斯特尼筛法 (Optimized Sieve of Eratosthenes)。
    该实现仅处理奇数，将内存占用减少约50%，并消除偶数检查的开销。

    参数:
        n (int): 一个正整数，表示查找素数的上限（包含n）。

    返回:
        list[int]: 一个包含所有小于等于n的素数的列表。
                   如果n小于2，则返回空列表。
    """
    if n < 2:
        return []

    # 特殊情况：如果上限小于3，直接返回 [2]
    if n == 2:
        return [2]

    # 仅处理奇数：
    # 索引 i 对应数值 2*i + 1
    # 数组大小：(n - 1) // 2，对应从 3 到 n 的奇数
    size = (n - 1) // 2
    is_prime = [True] * size

    # 只需筛选到 sqrt(n)
    limit = int(n**0.5)

    # 遍历奇数对应的索引
    # 对应的数值为 3, 5, 7...
    # 索引范围：从 1 (对应3) 到 (limit - 1) // 2
    for i in range(1, (limit - 1) // 2 + 1):
        if is_prime[i]:
            # 当前素数 p = 2*i + 1
            p = 2 * i + 1
            
            # 标记 p 的倍数（均为奇数）：p*p, p*(p+2)...
            # p*p 对应的索引：(p*p - 1) // 2
            # 步长：2*p (因为只标记奇数倍，偶数倍已隐式跳过)
            start_index = (p * p - 1) // 2
            
            # 使用切片赋值或循环标记
            # 这里使用显式循环，但在 Python 中，切片赋值通常更快
            # 但为了逻辑清晰和避免大步长切片带来的潜在内存问题，使用循环
            for j in range(start_index, size, p):
                is_prime[j] = False

    # 收集结果：[2] + [2*i+1 for i, prime in enumerate(is_prime) if prime]
    primes = [2] + [2 * i + 1 for i, prime in enumerate(is_prime) if prime]
    return primes

In [17]:
print(find_primes(100))

[2, 1, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


#### （3）案例参考

In [12]:
def find_primes(n):
    """
    Finds all prime numbers between 1 and n using the Sieve of Eratosthenes algorithm.

    :param n: The upper limit of the range to find prime numbers.
    :return: A list of all prime numbers between 1 and n.
    """
    if n < 2:
        return []

    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False

    p = 2
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * p, n + 1, p):
                is_prime[i] = False
        p += 1

    primes = [num for num in range(2, n + 1) if is_prime[num]]
    return primes

In [13]:
print(find_primes(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
